# Why we have to be hacky

Not our fault- colab ships an old (but not deprecated) Python that CLDK can't handle.

Next cell installs our utils package

Following cell sets up the env, you'll have to run this twice.

Cell after that checks that Py3.11 and Java are installed

In [1]:
!pip install git+https://github.com/ibm-granite-community/utils@1

  Cloning https://github.com/ibm-granite-community/utils (to revision 1) to /tmp/pip-req-build-xb6fu4sj
  Running command git clone --filter=blob:none --quiet https://github.com/ibm-granite-community/utils /tmp/pip-req-build-xb6fu4sj
  Running command git checkout -b 1 --track origin/1
  Switched to a new branch '1'
  Branch '1' set up to track remote branch '1' from 'origin'.
  Resolved https://github.com/ibm-granite-community/utils to commit ebd3a96c57d049b3ea52cb5bd202e1ea4e07e108
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.3/45.3 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 30.6 MB/s eta 0:00:00
  

In [2]:
!python --version
!java -version
!python -c "import numpy; print(numpy.__version__)"


Python 3.11.10
openjdk version "11.0.24" 2024-07-16
OpenJDK Runtime Environment (build 11.0.24+8-post-Ubuntu-1ubuntu322.04)
OpenJDK 64-Bit Server VM (build 11.0.24+8-post-Ubuntu-1ubuntu322.04, mixed mode, sharing)
2.1.1


In [6]:
!pip install git+https://github.com/ibm-granite-community/utils@1 --force

  Cloning https://github.com/ibm-granite-community/utils (to revision 1) to /tmp/pip-req-build-3ba6j8k7
  Running command git clone --filter=blob:none --quiet https://github.com/ibm-granite-community/utils /tmp/pip-req-build-3ba6j8k7
  Running command git checkout -b 1 --track origin/1
  Switched to a new branch '1'
  Branch '1' set up to track remote branch '1' from 'origin'.
  Resolved https://github.com/ibm-granite-community/utils to commit ebd3a96c57d049b3ea52cb5bd202e1ea4e07e108
  Preparing metadata (setup.py) ... done
  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
  Using cached langchain_community-0.3.0-py3-none-any.whl.metadata (2.8 kB)
  Using cached langchain_ollama-0.2.0-py3-none-any.whl.metadata (1.8 kB)
  Using cached replicate-0.33.0-py3-none-any.whl.metadata (25 kB)
  Using cached PyYAML-6.0.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.1 kB)
  Using cached SQLAlchemy-2.0.34-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_

In [1]:
#run this cell twice
from ibm_granite_community.colab_utils import setup_env_for_cldk
setup_env_for_cldk()

Upgrading Python to 3.11
You already have 3.11, nothing to install, proceeding.
Installing Java
removing and reinstalling numpy


sh: 1: Syntax error: Unterminated quoted string


### From this point on the notebook is the same

# Using CLDK to explain Java methods

In this tutorial, we will use [CLDK](https://github.com/IBM/codellm-devkit/tree/main) to generate a summary explaination for a Java method. You will explore some of the benefits of using CLDK to perform quick and easy program analysis and to build an LLM-based code summarizer. By the end of this tutorial, you will have implemented such a tool and generated a summary for a Java method.

Specifically, you will learn how to perform the following tasks on Java code to create LLM prompts for code summarization:

1. Create a new instance of the `CLDK` class.
2. Create an analysis object for the target Java code.
3. Iterate over all files in the code.
4. Iterate over all classes in a file.
5. Initialize `treesitter` utils for the class content.
6. Iterate over all methods in a class.
7. Get the code body for a method.
8. Sanitize the class for prompting the LLM.

We will write several helper methods to 1) format the LLM instruction for summarizing a given target method and 2) prompt the LLM via Ollama. We will then use CLDK to go through an application and generate the summary for the target method.

## Prequisites

Before we get started, let's make sure you have the following installed:

1. Python 3.11 or later (you can use [pyenv](https://github.com/pyenv/pyenv) to install Python)
2. Java 11 or later (you can use [SDKMAN!](https://sdkman.io) to install Java)
3. Maven 3.9 or later (you can use [SDKMAN!](https://sdkman.io) to install maven)
4. [Ollama 0.3.4](https://ollama.com/) or later.
5. [Granite code models](https://ollama.com/library/granite-code), which will serve as our LLM for this tutorial.

See the [Coding_Assistant_in_VSCode](../Coding_Assistant_in_VSCode/Coding_Assistant_in_VSCode.ipynb) recipe for instructions on setting up Ollama and installing the Granite models.

### Install and run Olamma (possibly not necessary)

In [2]:
!curl https://ollama.ai/install.sh | sh

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0>>> Installing ollama to /usr/local
100 13320    0 13320    0     0  52055      0 --:--:-- --:--:-- --:--:-- 52031
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [3]:
import os
os.system("nohup ollama serve &")

0

In [4]:
!ollama pull granite-code:3b

Couldn't find '/root/.ollama/id_ed25519'. Generating new private key.
Your new public key is: 

ssh-ed25519 AAAAC3NzaC1lZDI1NTE5AAAAIHDboz6O4b4fyW7DiVP+zw7KnkYwvGtoaNmEe4QqK9pw



2024/09/16 19:18:24 routes.go:1125: INFO server config env="map[CUDA_VISIBLE_DEVICES: GPU_DEVICE_ORDINAL: HIP_VISIBLE_DEVICES: HSA_OVERRIDE_GFX_VERSION: OLLAMA_DEBUG:false OLLAMA_FLASH_ATTENTION:false OLLAMA_GPU_OVERHEAD:0 OLLAMA_HOST:http://127.0.0.1:11434 OLLAMA_INTEL_GPU:false OLLAMA_KEEP_ALIVE:5m0s OLLAMA_LLM_LIBRARY: OLLAMA_LOAD_TIMEOUT:5m0s OLLAMA_MAX_LOADED_MODELS:0 OLLAMA_MAX_QUEUE:512 OLLAMA_MODELS:/root/.ollama/models OLLAMA_NOHISTORY:false OLLAMA_NOPRUNE:false OLLAMA_NUM_PARALLEL:0 OLLAMA_ORIGINS:[http://localhost https://localhost http://localhost:* https://localhost:* http://127.0.0.1 https://127.0.0.1 http://127.0.0.1:* https://127.0.0.1:* http://0.0.0.0 https://0.0.0.0 http://0.0.0.0:* https://0.0.0.0:* app://* file://* tauri://*] OLLAMA_RUNNERS_DIR: OLLAMA_SCHED_SPREAD:false OLLAMA_TMPDIR: ROCR_VISIBLE_DEVICES:]"
time=2024-09-16T19:18:24.917Z level=INFO source=images.go:753 msg="total blobs: 0"
time=2024-09-16T19:18:24.918Z level=INFO source=images.go:760 msg="total unu

[GIN] 2024/09/16 - 19:19:05 | 200 |      66.561µs |       127.0.0.1 | HEAD     "/"
pulling manifest ⠙ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ 

time=2024-09-16T19:19:06.778Z level=INFO source=download.go:175 msg="downloading 9699e81546fa in 16 124 MB part(s)"


pulling manifest 
pulling 9699e81546fa...   0% ▕▏    0 B/2.0 GB                  pulling manifest 
pulling 9699e81546fa...   0% ▕▏    0 B/2.0 GB                  pulling manifest 
pulling 9699e81546fa...   0% ▕▏ 7.0 MB/2.0 GB                  pulling manifest 
pulling 9699e81546fa...   1% ▕▏  20 MB/2.0 GB                  pulling manifest 
pulling 9699e81546fa...   2% ▕▏  46 MB/2.0 GB                  pulling manifest 
pulling 9699e81546fa...   4% ▕▏  79 MB/2.0 GB                  pulling manifest 
pulling 9699e81546fa...   5% ▕▏ 109 MB/2.0 GB                  pulling manifest 
pulling 9699e81546fa...   6% ▕▏ 121 MB/2.0 GB                  pulling manifest 
pulling 9699e81546fa...   7% ▕▏ 140 MB/2.0 GB                  pulling manifest 
pulling 9699e81546fa...   8% ▕▏ 154 MB/2.0 GB                  pulling manifest 
pulling 9699e81546fa...   8% ▕▏ 162 MB/2.0 GB  162 MB/s     11spulling manifest 
pulling 9699e81546fa...  10% ▕▏ 190 MB/2.0 GB  162 MB/s     11spulling manifest 
pulling 96

...

pulling manifest 
pulling 9699e81546fa... 100% ▕▏ 2.0 GB                         
pulling 977871d28ce4... 100% ▕▏  679 B                         
pulling 43070e2d4e53...   0% ▕▏    0 B/ 11 KB                  pulling manifest 
pulling 9699e81546fa... 100% ▕▏ 2.0 GB                         
pulling 977871d28ce4... 100% ▕▏  679 B                         
pulling 43070e2d4e53... 100% ▕▏  11 KB                         pulling manifest 
pulling 9699e81546fa... 100% ▕▏ 2.0 GB                         
pulling 977871d28ce4... 100% ▕▏  679 B                         
pulling 43070e2d4e53... 100% ▕▏  11 KB                         pulling manifest 
pulling 9699e81546fa... 100% ▕▏ 2.0 GB                         
pulling 977871d28ce4... 100% ▕▏  679 B                         
pulling 43070e2d4e53... 100% ▕▏  11 KB                         pulling manifest 
pulling 9699e81546fa... 100% ▕▏ 2.0 GB                         
pulling 977871d28ce4... 100% ▕▏  679 B                         
pulling 43070e2d4e

time=2024-09-16T19:19:29.129Z level=INFO source=download.go:175 msg="downloading f1406f0e20d0 in 1 43 B part(s)"


pulling manifest 
pulling 9699e81546fa... 100% ▕▏ 2.0 GB                         
pulling 977871d28ce4... 100% ▕▏  679 B                         
pulling 43070e2d4e53... 100% ▕▏  11 KB                         
pulling f1406f0e20d0...   0% ▕▏    0 B/  43 B                  pulling manifest 
pulling 9699e81546fa... 100% ▕▏ 2.0 GB                         
pulling 977871d28ce4... 100% ▕▏  679 B                         
pulling 43070e2d4e53... 100% ▕▏  11 KB                         
pulling f1406f0e20d0...   0% ▕▏    0 B/  43 B                  pulling manifest 
pulling 9699e81546fa... 100% ▕▏ 2.0 GB                         
pulling 977871d28ce4... 100% ▕▏  679 B                         
pulling 43070e2d4e53... 100% ▕▏  11 KB                         
pulling f1406f0e20d0... 100% ▕▏   43 B                         pulling manifest 
pulling 9699e81546fa... 100% ▕▏ 2.0 GB                         
pulling 977871d28ce4... 100% ▕▏  679 B                         
pulling 43070e2d4e53... 100% ▕▏  11

time=2024-09-16T19:19:31.082Z level=INFO source=download.go:175 msg="downloading 1e60cb6006a8 in 1 559 B part(s)"


pulling manifest 
pulling 9699e81546fa... 100% ▕▏ 2.0 GB                         
pulling 977871d28ce4... 100% ▕▏  679 B                         
pulling 43070e2d4e53... 100% ▕▏  11 KB                         
pulling f1406f0e20d0... 100% ▕▏   43 B                         
pulling 1e60cb6006a8...   0% ▕▏    0 B/ 559 B                  pulling manifest 
pulling 9699e81546fa... 100% ▕▏ 2.0 GB                         
pulling 977871d28ce4... 100% ▕▏  679 B                         
pulling 43070e2d4e53... 100% ▕▏  11 KB                         
pulling f1406f0e20d0... 100% ▕▏   43 B                         
pulling 1e60cb6006a8... 100% ▕▏  559 B                         pulling manifest 
pulling 9699e81546fa... 100% ▕▏ 2.0 GB                         
pulling 977871d28ce4... 100% ▕▏  679 B                         
pulling 43070e2d4e53... 100% ▕▏  11 KB                         
pulling f1406f0e20d0... 100% ▕▏   43 B                         
pulling 1e60cb6006a8... 100% ▕▏  559 B              

### Install Ollama Python SDK

We also need the Python API for Ollama and the CLDK toolkit.

In [5]:
!pip install ollama

### Install CLDK
CLDK is avaliable at https://github.com/IBM/codellm-devkit. You can install it by running the following command:

In [6]:
!pip install git+https://github.com/IBM/codellm-devkit.git

  Cloning https://github.com/IBM/codellm-devkit.git to /tmp/pip-req-build-5hogm27z
  Running command git clone --filter=blob:none --quiet https://github.com/IBM/codellm-devkit.git /tmp/pip-req-build-5hogm27z
  Resolved https://github.com/IBM/codellm-devkit.git to commit ac657363e68d3c09024949a8d5f66a25adc7597d
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/tree-sitter/tree-sitter-python (to revision 0f9047c) to /tmp/pip-install-gy4d_90u/tree-sitter-python_9fb3008645de4882ab4a09cfd14564f3
  Running command git clone --filter=blob:none --quiet https://github.com/tree-sitter/tree-sitter-python /tmp/pip-install-gy4d_90u/tree-sitter-python_9fb3008645de4882ab4a09cfd14564f3
  Running command git checkout -q 0f9047c
  Resolved https://github.com/tree-sitter/tree-sitter-python to commit 0f9047c
  Installing build dependencies ... done
  Getting requirements to build wheel ... don

## Analyze and Summarize Java Code

First we get the sample Java code, then we will use CLDK to analyze and summarize it.

### Get the Sample Java Code
For this tutorial, we will use [Apache Commons CLI](https://github.com/apache/commons-cli) as the sample Java code. You can download the source code to a temporary directory by running the following commands:

In [7]:
%%bash
COMMONS=commons-cli-1.7.0
test -d temp || ( \
  mkdir -p temp && \
  cd temp && \
  wget https://github.com/apache/commons-cli/archive/refs/tags/rel/$COMMONS.zip -O $COMMONS.zip && \
  unzip -o $COMMONS.zip && \
  rm -f $COMMONS.zip && \
  cd - \
) && ls -l temp/

--2024-09-16 19:20:28--  https://github.com/apache/commons-cli/archive/refs/tags/rel/commons-cli-1.7.0.zip
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/apache/commons-cli/zip/refs/tags/rel/commons-cli-1.7.0 [following]
--2024-09-16 19:20:28--  https://codeload.github.com/apache/commons-cli/zip/refs/tags/rel/commons-cli-1.7.0
Resolving codeload.github.com (codeload.github.com)... 140.82.112.9
Connecting to codeload.github.com (codeload.github.com)|140.82.112.9|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘commons-cli-1.7.0.zip’

     0K .......... .......... .......... .......... .......... 2.00M
    50K .......... .......... .......... .......... .......... 3.98M
   100K .......... .......... .......... .......... .......... 42.3M
   150K .......... .......

Archive:  commons-cli-1.7.0.zip
caed6714a73ce366aebccebf4a21e287d6d34ae0
   creating: commons-cli-rel-commons-cli-1.7.0/
  inflating: commons-cli-rel-commons-cli-1.7.0/.asf.yaml  
   creating: commons-cli-rel-commons-cli-1.7.0/.github/
  inflating: commons-cli-rel-commons-cli-1.7.0/.github/GH-ROBOTS.txt  
  inflating: commons-cli-rel-commons-cli-1.7.0/.github/dependabot.yml  
   creating: commons-cli-rel-commons-cli-1.7.0/.github/workflows/
  inflating: commons-cli-rel-commons-cli-1.7.0/.github/workflows/codeql-analysis.yml  
  inflating: commons-cli-rel-commons-cli-1.7.0/.github/workflows/coverage.yml  
  inflating: commons-cli-rel-commons-cli-1.7.0/.github/workflows/maven.yml  
  inflating: commons-cli-rel-commons-cli-1.7.0/.github/workflows/scorecards-analysis.yml  
  inflating: commons-cli-rel-commons-cli-1.7.0/.gitignore  
  inflating: commons-cli-rel-commons-cli-1.7.0/CODE_OF_CONDUCT.md  
  inflating: commons-cli-rel-commons-cli-1.7.0/CONTRIBUTING.md  
  inflating: commons-cli-re

You should see `commons-cli-rel-commons-cli-1.7.0` in `temp`.

### Generate the Code Summary

Code summarization or code explanation is the task of converting code written in a programming language to natural language. It has several benefits, such as understanding code without looking at its details, documenting code for better maintenance, etc. To perform code summarization, one needs to understand the basic details of the code implementation, and use that knowledge to generate the summary using various AI-based approaches. Here, we use LLMs, specifically Granite Code 83 Instruct. We will show how a developer can easily use CLDK to analyze code by calling various APIs without having to implement analyses with lower-level tools.

#### Step 1: Add the neccessary imports

In [8]:
!pip uninstall -y numpy
!pip install numpy=="1.26.4"

Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4
  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.3 MB)


#maybe you just have to always restart just before this cell in Colab?

In [9]:
import ollama
from cldk import CLDK
from cldk.analysis import AnalysisLevel

#### Step 2: Define a function for creating the LLM prompt

This function instructs the LLM to summarize a Java method and includes relevant code for the task.

In [10]:
def format_inst(code, focal_method, focal_class, language):
    """
    Format the instruction for the given focal method and class.
    """
    inst = f"Question: Can you write a brief summary for the method `{focal_method}` in the class `{focal_class}` below?\n"

    inst += "\n"
    inst += f"```{language}\n"
    inst += code
    inst += "```" if code.endswith("\n") else "\n```"
    inst += "\n"
    return inst

#### Step 3: Define a function to call the LLM

In this case, Granite Code 3b Instruct, invoked using Ollama.

In [16]:
def prompt_ollama(message: str, model_id: str = "granite-code:3b") -> str:
    """Prompt local model on Ollama"""
    response_object = ollama.generate(model=model_id, prompt=message, options={"temperature":0.2})
    return response_object["response"]

#### Step 4: Create an instance of CLDK, specifying the programming language of the source code

Java in this case.

In [17]:
cldk = CLDK(language="java")

#### Step 5: Select the analysis engine and analysis level

CLDK uses different analysis engines---[CodeAnalyzer](https://github.com/IBM/codenet-minerva-code-analyzer) (built over [WALA](https://github.com/wala/WALA) and [JavaParser](https://github.com/javaparser/javaparser)), [Treesitter](https://tree-sitter.github.io/tree-sitter/), and [CodeQL](https://codeql.github.com/) (future). CodeAnalyzer is the default analysis engine. CLDK supports different analysis levels: 1) symbol table, 2) call graph, 3) program dependency graph, and 4) system dependency graph. The analysis level can be selected using the `AnalysisLevel` enumerated type. For this example, we select the symbol-table analysis level, with CodeAnalyzer as the default analysis engine.

In [18]:
# Create an analysis object for the Java application
analysis = cldk.analysis(project_path="temp/commons-cli-rel-commons-cli-1.7.0", analysis_level=AnalysisLevel.symbol_table, analysis_json_path='analysis')

In [19]:
!cat analysis/analysis.json

Streaming output truncated to the last 5000 lines.
                  "is_static_call": false,
                  "is_constructor_call": false,
                  "start_line": 247,
                  "start_column": 17,
                  "end_line": 247,
                  "end_column": 46
                },
                {
                  "method_name": "processPropertiesFromValues",
                  "receiver_expr": "",
                  "receiver_type": "",
                  "argument_types": [
                    "java.util.Properties",
                    "java.util.List<java.lang.String>"
                  ],
                  "return_type": "",
                  "callee_signature": "processPropertiesFromValues(java.util.Properties, java.util.List<java.lang.String>)",
                  "is_public": false,
                  "is_protected": false,
                  "is_private": true,
                  "is_unspecified": false,
                  "is_static_call": false,
           

Note that a file [./analysis/analysis.json](./analysis/analysis.json) was created. This file will be used in the other notebooks, too!

#### Step 6: Iterate over all the class files and create the prompt

In this case, we want to provide a sanitized Java class in the prompt, containing only the relevant information for summarizing the target method. To illustrate, consider the floowing class:

```java
package com.ibm.org;
import A.B.C.D;
...
public class Foo {
 // code comment
 public void bar(){
    int a;
    a = baz();
    // do something
    }
 private int baz()
 {
    // do something
 }
 public String dummy (String a)
 {
    // do somthing
 }
```
Let's say we want to generate a summary for method `bar`. To understand what it does, we add the callees of this method in the prompt, which in this case includes `baz`. We remove the other methods, imports, comments, etc. All of this can be achieved with a single call to CLDK's `sanitize_focal_class` API. In this process, we also use Treesitter to analyze the code.  After creating the sanitized code, we call the previously defined `format_inst` method to create the LLM prompt and pass the prompt to `prompt_ollama` to generate the method summary.

In [20]:
# For simplicity, we run the code summarization on a single class and method
# (this filter can be removed to run this code over the entire application)
target_class = "org.apache.commons.cli.GnuParser"
target_method = "flatten(Options, String[], boolean)"

# Iterate over all classes in the application
for class_name in analysis.get_classes():
    if class_name == target_class:
        print(f"Class: {class_name}")
        class_file_path = analysis.get_java_file(qualified_class_name=class_name)

        # Read code for the class
        with open(class_file_path, "r") as f:
            code_body = f.read()

        # Initialize treesitter utils for the class file content
        tree_sitter_utils = cldk.tree_sitter_utils(source_code=code_body)

        # Iterate over all methods in class
        for method in analysis.get_methods_in_class(qualified_class_name=class_name):
            if method == target_method:

                # Get all the method details
                method_details = analysis.get_method(qualified_class_name=class_name,
                                                     qualified_method_name=method)

                # Sanitize the class for analysis with respect to the target method
                sanitized_class = tree_sitter_utils.sanitize_focal_class(method_details.declaration)

                # Format the instruction for the given target method and class
                instruction = format_inst(
                    code=sanitized_class,
                    focal_method=method_details.declaration,
                    focal_class=class_name.split(".")[-1],
                    language="java"
                )

                print(f"Instruction:\n{instruction}\n")
                print(f"Generating code summary and it will take few minutes (or even seconds) based on where the model has been hosted...\n")

                # Prompt the local model on Ollama
                llm_output = prompt_ollama(message=instruction)

                # Print the LLM output
                print(f"LLM Output:\n{llm_output}")

Class: org.apache.commons.cli.GnuParser
Instruction:
Question: Can you write a brief summary for the method `protected String[] flatten(final Options options, final String[] arguments, final boolean stopAtNonOption)` in the class `GnuParser` below?

```java
package org.apache.commons.cli;

import java.util.ArrayList;
import java.util.List;

@Deprecated
public class GnuParser extends Parser {

    @Override
    protected String[] flatten(final Options options, final String[] arguments, final boolean stopAtNonOption) {
        final List<String> tokens = new ArrayList<>();
        boolean eatTheRest = false;
        for (int i = 0; i < arguments.length; i++) {
            final String arg = arguments[i];
            if ("--".equals(arg)) {
                eatTheRest = true;
                tokens.add("--");
            } else if ("-".equals(arg)) {
                tokens.add("-");
            } else if (arg.startsWith("-")) {
                final String opt = Util.stripLeadingHyphens(ar

time=2024-09-16T19:27:52.387Z level=INFO source=server.go:101 msg="system memory" total="12.7 GiB" free="11.2 GiB" free_swap="0 B"
time=2024-09-16T19:27:52.394Z level=INFO source=memory.go:326 msg="offload to cpu" layers.requested=-1 layers.model=33 layers.offload=0 layers.split="" memory.available="[11.2 GiB]" memory.gpu_overhead="0 B" memory.required.full="5.0 GiB" memory.required.partial="0 B" memory.required.kv="2.5 GiB" memory.required.allocations="[5.0 GiB]" memory.weights.total="4.3 GiB" memory.weights.repeating="4.2 GiB" memory.weights.nonrepeating="98.4 MiB" memory.graph.full="548.0 MiB" memory.graph.partial="621.5 MiB"
time=2024-09-16T19:27:52.395Z level=INFO source=server.go:391 msg="starting llama server" cmd="/tmp/ollama2323584229/runners/cpu_avx2/ollama_llama_server --model /root/.ollama/models/blobs/sha256-9699e81546fad30bfcc6a005f9c30316828b444e37ece48ddab1babfb4d634c5 --ctx-size 8192 --batch-size 512 --embedding --log-disable --no-mmap --parallel 4 --port 44003"
time=2

INFO [main] build info | build=1 commit="8962422" tid="133186671169664" timestamp=1726514872
INFO [main] system info | n_threads=1 n_threads_batch=1 system_info="AVX = 1 | AVX_VNNI = 0 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | AVX512_BF16 = 0 | FMA = 1 | NEON = 0 | SVE = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | MATMUL_INT8 = 0 | LLAMAFILE = 1 | " tid="133186671169664" timestamp=1726514872 total_threads=2
INFO [main] HTTP server listening | hostname="127.0.0.1" n_threads_http="6" port="44003" tid="133186671169664" timestamp=1726514872


llama_model_loader: - kv  24:                      tokenizer.ggml.tokens arr[str,49152]   = ["<|endoftext|>", "<fim_prefix>", "<f...
llama_model_loader: - kv  25:                  tokenizer.ggml.token_type arr[i32,49152]   = [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, ...
time=2024-09-16T19:27:52.652Z level=INFO source=server.go:624 msg="waiting for server to become available" status="llm server loading model"
llama_model_loader: - kv  26:                      tokenizer.ggml.merges arr[str,48891]   = ["Ġ Ġ", "ĠĠ ĠĠ", "ĠĠĠĠ ĠĠ...
llama_model_loader: - kv  27:                tokenizer.ggml.bos_token_id u32              = 0
llama_model_loader: - kv  28:                tokenizer.ggml.eos_token_id u32              = 0
llama_model_loader: - kv  29:            tokenizer.ggml.unknown_token_id u32              = 0
llama_model_loader: - kv  30:            tokenizer.ggml.padding_token_id u32              = 0
llama_model_loader: - kv  31:                    tokenizer.chat_template str              = {% f

INFO [main] model loaded | tid="133186671169664" timestamp=1726514884


time=2024-09-16T19:28:04.932Z level=INFO source=server.go:624 msg="waiting for server to become available" status="llm server not responding"
time=2024-09-16T19:28:05.184Z level=INFO source=server.go:629 msg="llama runner started in 12.78 seconds"


LLM Output:
The `flatten` method is responsible for parsing command line arguments and flattening them into a list of tokens. It takes three parameters: `options`, which represents the available options; `arguments`, which contains the command line arguments to be parsed; and `stopAtNonOption`, which determines whether the parser should stop at non-option arguments or not.

The method iterates through each argument in the `arguments` array and checks if it is an option or a non-option argument. If it is an option, it checks if it exists in the `options` object. If it does, it adds the option to the `tokens` list. If it doesn't exist, it checks for an equal sign (`=`) in the option and determines whether the option has an associated value. If so, it adds the option and its value to the `tokens` list. If the option doesn't have a value, it adds the option to the `tokens` list.

If the argument is not an option, it is added to the `tokens` list. If the `stopAtNonOption` parameter is set t

After the LLM's response is received, you should see the generated summary of the `flatten` method printed out (similar to the following text).


**LLM Output:**
Sure! The method `protected String[] flatten(final Options options, final String[] arguments, final boolean stopAtNonOption)` in the class `GnuParser` is responsible for parsing command-line arguments according to the GNU convention. It takes three parameters: `options`, which represents a collection of valid command-line options; `arguments`, which is an array of strings representing the command-line arguments to be parsed; and `stopAtNonOption`, which determines whether to stop processing arguments once a non-option argument is encountered.

The method iterates through each argument in the `arguments` array and performs various checks based on the argument's format. If the argument starts with "--", it adds the entire argument to the `tokens` list. If the argument is "-", it adds "-" to the `tokens` list. If the argument starts with a hyphen but does not match any known option, it determines whether to stop processing arguments or add the argument to the `tokens` list based on the value of `stopAtNonOption`.

If an argument matches a known option, it is added to the `tokens` list along with its associated value (if applicable). The method also handles cases where the argument appears in a different format, such as "-Dproperty=value" or "--option=value".

Finally, the method returns the `tokens` list as an array of strings.
